# Preprocessing and Modeling

Our data has been cleaned, most features have been selected, and our outliers have been removed. Most of the variables, if not all of the variables, have some relatively strong relationship with sale price. Now we can begin identifying a model that will perform up to par.

## Data and Package Imports

In [178]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings("ignore")

In [179]:
df_train = pd.read_csv('../data/train_clean.csv')
df_test = pd.read_csv('../data/test.csv')

## Train_Test_Split and Separating Features

In [180]:
# Separating numeric and categorical columns to be used in one-hot encoding
numeric = [col for col in df_train._get_numeric_data().columns if not col in ['id','sale_price']]
categorical = [col for col in df_train.columns if col not in numeric and col not in ['id','sale_price']]

In [181]:
X = df_train.drop(columns=['id','sale_price'])
y = df_train['sale_price']

In [182]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=13
)

## Pipelines and Model Testing

### Scaling and Encoding

In [183]:
# Column Transformer made to scale input data and encode categorical data.
# Can apply to all models. This separates numeric and categorical variables for model fit. In theory, only needs to be instantiated once.

ctx = ColumnTransformer(
    [
    ('ss', StandardScaler(), numeric),
    ('ohe', OneHotEncoder(
        handle_unknown='ignore',
        drop='first'),
        # Drops the first categorical variable, and all coefficients are to be interpreted in relation to the dropped variable
        # For 'neighborhood': dropped variable is 'Sawyer'
        # For 'home_type': dropped variable is 'Newer 2 Story' (2 story home built in 1946 or newer)
        categorical)
    ]
) 

### Instantiating Models

In [184]:
# Linear Regression Pipeline
lr_pipe = Pipeline([
    ('ctx', ctx),
    ('lr', TransformedTargetRegressor(LinearRegression(), func=np.log, inverse_func=np.exp))
])

# Ridge Regression Pipeline
rg_pipe = Pipeline([
    ('ctx', ctx),
    ('rg', TransformedTargetRegressor(Ridge(), func=np.log, inverse_func=np.exp))
])

# Lasso Regression Pipeline
lasso_pipe = Pipeline([
    ('ctx', ctx),
    ('lasso', TransformedTargetRegressor(Lasso(), func=np.log, inverse_func=np.exp))
])

# ElasticNet Pipeline
enet_pipe = Pipeline([
    ('ctx', ctx),
    ('enet', TransformedTargetRegressor(ElasticNet(), func=np.log, inverse_func=np.exp))
])

# KNeighborsRegressor Pipeline
knn_pipe = Pipeline([
    ('ctx', ctx),
    ('knn', TransformedTargetRegressor(KNeighborsRegressor(), func=np.log, inverse_func=np.exp))
])

### Model RMSE Comparison

In [185]:
# Calculate null predictions to compare our models to
y_preds_null = np.full_like(y, y.mean())

In [186]:
# Fitting models
lr_pipe.fit(X_train, y_train)
rg_pipe.fit(X_train, y_train)
lasso_pipe.fit(X_train, y_train)
enet_pipe.fit(X_train, y_train)
knn_pipe.fit(X_train, y_train)

Pipeline(steps=[('ctx',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  ['home_quality', 'home_sqft',
                                                   'garage_cars', 'home_age',
                                                   'yr_remod', 'full_bath',
                                                   'masonry_veneer_area',
                                                   'total_rooms_above_ground']),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['neighborhood',
                                                   'home_type'])])),
                ('knn',
                 TransformedTargetRegressor(func=<ufunc 'log'>,
                                            inverse_func=<ufunc 

Here we are comparing out model RMSEs to the Null RMSE to see if the error of our models are acceptable.

In [187]:
print('Null RMSE:', round(np.sqrt(mean_squared_error(y, y_preds_null)), 2))
print('')
print('Linear Regression RMSE:', round(np.sqrt(mean_squared_error(y_train, lr_pipe.predict(X_train))), 2))
print('Ridge RMSE:', round(np.sqrt(mean_squared_error(y_train, rg_pipe.predict(X_train))), 2))
print('Lasso RMSE:', round(np.sqrt(mean_squared_error(y_train, lasso_pipe.predict(X_train))), 2))
print('ElasticNet RMSE:', round(np.sqrt(mean_squared_error(y_train, enet_pipe.predict(X_train))), 2))
print('KNearestRegressor RMSE:', round(np.sqrt(mean_squared_error(y_train, knn_pipe.predict(X_train))), 2))

Null RMSE: 65897.4

Linear Regression RMSE: 20387.94
Ridge RMSE: 20418.25
Lasso RMSE: 66625.75
ElasticNet RMSE: 66625.75
KNearestRegressor RMSE: 20034.5


At the moment, judging solely by RMSE, the only usable models are Linear Regression, Ridge, and KNearestRegressor. The best model based on RMSE is KNearestRegressor and the worst models are ElasticNet and Lasso. We can also use the disparity between training and testing scores for these models to measure their effectiveness in generating predictions and generating insights on individual feature effect on sale price.

### Model R2 / Accuracy Score Comparison

In [188]:
print('Linear Regression Training Score:', round(cross_val_score(lr_pipe, X_train, y_train).mean(), 3))
print('Linear Regression Validation Score:', round(cross_val_score(lr_pipe, X_val, y_val).mean(), 3))
print("")
print('Ridge Training Score:', round(cross_val_score(rg_pipe, X_train, y_train).mean(), 3))
print('Ridge Validation Score:', round(cross_val_score(rg_pipe, X_val, y_val).mean(), 3))
print("")
print('Lasso Training Score:', round(cross_val_score(lasso_pipe, X_train, y_train).mean(), 3))
print('Lasso Validation Score:', round(cross_val_score(lasso_pipe, X_val, y_val).mean(), 3))
print("")
print('ElasticNet Training Score:', round(cross_val_score(enet_pipe, X_train, y_train).mean(), 3))
print('ElasticNet Validation Score:', round(cross_val_score(enet_pipe, X_val, y_val).mean(), 3))
print("")
print('KNearestRegressor Training Score:', round(cross_val_score(knn_pipe, X_train, y_train).mean(), 3))
print('KNearestRegressor Validation Score:', round(cross_val_score(knn_pipe, X_val, y_val).mean(), 3))

Linear Regression Training Score: 0.893
Linear Regression Validation Score: 0.857

Ridge Training Score: 0.893
Ridge Validation Score: 0.863

Lasso Training Score: -0.037
Lasso Validation Score: -0.047

ElasticNet Training Score: -0.037
ElasticNet Validation Score: -0.047

KNearestRegressor Training Score: 0.849
KNearestRegressor Validation Score: 0.782


For the first 4 scores, these scores (R^2) are an evaluator as to how much of the data's variance they can account for. For KNearestRegressor, its score is accuracy. Based on these scores, every model is within a 3% threshold of underfit/overfit, but are slightly overfit. Our models are slightly worse at predicting with new data than predicting y_train. This is likely due to the large amount of features, and small amount of observations. The most overfit of all of the models is KNearestRegressor. Based solely on scores, the best model is Ridge and the worst model is ElasticNet.

We can use the coefficients from Ridge to interpret the effects that our features have on home sale price.

In [189]:
data = {
    'Features' : ctx.get_feature_names_out(),
    'Coefficient' : np.exp(rg_pipe.named_steps.rg.regressor_.coef_) - 1 # Need to do a .exp() and -1 because I used the log of my y variable to make the coefficients interpretable.
}

coefficients = pd.DataFrame(data).sort_values(
                by="Coefficient",
                ascending=False, # largest first
                key=abs) # Sorted by absolute value

coefficients.head(10) # top 10 coefficient values

,Features,Coefficient
1,ss__home_sqft,0.173751
13,ohe__neighborhood_Crawfor,0.163102
11,ohe__neighborhood_ClearCr,0.132807
33,ohe__neighborhood_Veenker,0.120682
43,ohe__home_type_Split Foyer,0.114954
37,ohe__home_type_2 Story PUD,-0.114078
17,ohe__neighborhood_IDOTRR,-0.107370
0,ss__home_quality,0.102906
9,ohe__neighborhood_BrDale,-0.084809
38,ohe__home_type_Duplex,-0.079169


Because we used the log of our y to normalize sale price, the way that we interpret our coefficients is different from how it usually is with regression-based models. For every 1 unit change in home square footage, assuming all else constant, home price increases by 17%. To use a categorical variable as an example, if two homes were in different neighborhoods, assuming all else constant, the price of a home in Crawford would be worth 15% more than the price of a home in Sawyer. It is apparent that the strongest variables in our feature set are home square footage, home type (originally MS Subclass), neighborhood, and home quality.

## Implementation of Polynomial Features

In [190]:
# Creating a new instance of ctx to include polynomial features and only apply polynomial features and standardscaler
pfs = Pipeline([
    ('pf', PolynomialFeatures(
        degree=2,
        interaction_only=True,
        include_bias=False
    )),
    ('sc', StandardScaler()),
])

# Separating polynomial features and standardscaler from column transformer so that standardscaler can scale polynomial features and all numeric features

ctx_poly = ColumnTransformer([
    ('pfs', pfs, numeric),
    ('ohe', OneHotEncoder(
        handle_unknown='ignore',
        drop='first'
        # Drops the first categorical variable, and all coefficients are to be interpreted in relation to the dropped variable
        # For 'neighborhood': dropped variable is 'Sawyer'
        # For 'home_type': dropped variable is 'Newer 2 Story' (2 story home built in 1946 or newer)
    ), categorical)
])

In [191]:
# Not applying TransformedTargetRegressor here, after some testing it doesn't seem to affect RMSE or R^2 scores when applied with polynomial features.

# Linear Regression Pipeline
lr_pipe_poly = Pipeline([
    ('ctx', ctx_poly),
    ('lr', LinearRegression())
])

# Ridge Regression Pipeline
rg_pipe_poly = Pipeline([
    ('ctx', ctx_poly),
    ('rg', Ridge())
])

# Lasso Regression Pipeline
lasso_pipe_poly = Pipeline([
    ('ctx', ctx_poly),
    ('lasso', Lasso())
])

# ElasticNet Pipeline
enet_pipe_poly = Pipeline([
    ('ctx', ctx_poly),
    ('enet', ElasticNet())
])

# KNeighborsRegressor Pipeline
knn_pipe_poly = Pipeline([
    ('ctx', ctx_poly),
    ('knn', KNeighborsRegressor())
])

In [192]:
# Fitting models again, this time including polynomial features
lr_pipe_poly.fit(X_train, y_train)
rg_pipe_poly.fit(X_train, y_train)
lasso_pipe_poly.fit(X_train, y_train)
enet_pipe_poly.fit(X_train, y_train)
knn_pipe_poly.fit(X_train, y_train)

Pipeline(steps=[('ctx',
                 ColumnTransformer(transformers=[('pfs',
                                                  Pipeline(steps=[('pf',
                                                                   PolynomialFeatures(include_bias=False,
                                                                                      interaction_only=True)),
                                                                  ('sc',
                                                                   StandardScaler())]),
                                                  ['home_quality', 'home_sqft',
                                                   'garage_cars', 'home_age',
                                                   'yr_remod', 'full_bath',
                                                   'masonry_veneer_area',
                                                   'total_rooms_above_ground']),
                                                 ('ohe',
                      

### Model RMSE Comparison - Polynomials

In [193]:
print('Null RMSE:', round(np.sqrt(mean_squared_error(y, y_preds_null)), 2))
print('')
print('Linear Regression RMSE:', round(np.sqrt(mean_squared_error(y_train, lr_pipe_poly.predict(X_train))), 2))
print('Ridge RMSE:', round(np.sqrt(mean_squared_error(y_train, rg_pipe_poly.predict(X_train))), 2))
print('Lasso RMSE:', round(np.sqrt(mean_squared_error(y_train, lasso_pipe_poly.predict(X_train))), 2))
print('ElasticNet RMSE:', round(np.sqrt(mean_squared_error(y_train, enet_pipe_poly.predict(X_train))), 2))
print('KNearestRegressor RMSE:', round(np.sqrt(mean_squared_error(y_train, knn_pipe_poly.predict(X_train))), 2))

Null RMSE: 65897.4

Linear Regression RMSE: 19839.22
Ridge RMSE: 20000.34
Lasso RMSE: 19992.44
ElasticNet RMSE: 24301.74
KNearestRegressor RMSE: 20016.61


At the moment, judging solely by RMSE, all of our models are still valid and can identify relationships and produce useful predictions from polynomial data. The best model is LinearRegression, and the worst model is ElasticNet.

### Model R^2 / Accuracy Score Comparison - Polynomials

In [194]:
print('Linear Regression Training Score:', round(cross_val_score(lr_pipe_poly, X_train, y_train).mean(), 3))
print('Linear Regression Validation Score:', round(cross_val_score(lr_pipe_poly, X_val, y_val).mean(), 3))
print("")
print('Ridge Training Score:', round(cross_val_score(rg_pipe_poly, X_train, y_train).mean(), 3))
print('Ridge Validation Score:', round(cross_val_score(rg_pipe_poly, X_val, y_val).mean(), 3))
print("")
print('Lasso Training Score:', round(cross_val_score(lasso_pipe_poly, X_train, y_train).mean(), 3))
print('Lasso Validation Score:', round(cross_val_score(lasso_pipe_poly, X_val, y_val).mean(), 3))
print("")
print('ElasticNet Training Score:', round(cross_val_score(enet_pipe_poly, X_train, y_train).mean(), 3))
print('ElasticNet Validation Score:', round(cross_val_score(enet_pipe_poly, X_val, y_val).mean(), 3))
print("")
print('KNearestRegressor Training Score:', round(cross_val_score(knn_pipe_poly, X_train, y_train).mean(), 3))
print('KNearestRegressor Validation Score:', round(cross_val_score(knn_pipe_poly, X_val, y_val).mean(), 3))

Linear Regression Training Score: 0.893
Linear Regression Validation Score: 0.832

Ridge Training Score: 0.894
Ridge Validation Score: 0.847

Lasso Training Score: 0.893
Lasso Validation Score: 0.835

ElasticNet Training Score: 0.859
ElasticNet Validation Score: 0.827

KNearestRegressor Training Score: 0.848
KNearestRegressor Validation Score: 0.776


Based on these scores, every model is still slightly overfit, potentially more overfit. Our models are slightly worse at predicting with new data than predicting y_train. This is likely due to the large amount of features, and small amount of observations. Based solely on scores, the best model is Ridge and the worst model is KNearestRegressor.

## GridSearchCV for Optimization of Ridge - Original

Since the non-Polynomial Ridge model RMSE was very close to the Polynomial scores, for the sake of coefficient interpretability, I am going to use the original Ridge model as my final model for this project.

In [195]:
rg_params = {
    'rg__regressor__alpha' : [1, 2, 3, 4, 5],
    'rg__regressor__random_state' : [1, 2, 3, 4, 5],
    'rg__regressor__solver' : ['auto', 'sag', 'lbfgs']
}

In [196]:
gs = GridSearchCV(
    rg_pipe,
    rg_params,
    cv = 5,
    scoring='neg_mean_squared_error'
)

In [197]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ctx',
                                        ColumnTransformer(transformers=[('ss',
                                                                         StandardScaler(),
                                                                         ['home_quality',
                                                                          'home_sqft',
                                                                          'garage_cars',
                                                                          'home_age',
                                                                          'yr_remod',
                                                                          'full_bath',
                                                                          'masonry_veneer_area',
                                                                          'total_rooms_above_ground']),
                                      

In [198]:
np.sqrt(-gs.best_score_) # Best RMSE score of GridSearch optimized Ridge regression model

21331.47560514137

In [199]:
gs.best_params_ # GridSearch returned best parameters for Ridge regression model

{'rg__regressor__alpha': 4,
 'rg__regressor__random_state': 3,
 'rg__regressor__solver': 'sag'}

After some testing, I've found that the best Ridge regression parameters are the default ones.

## Final Model Fitting - X_Val

In [200]:
# Final Ridge Regression Pipeline with default parameters
rg_pipe = Pipeline([
    ('ctx', ctx),
    ('rg', TransformedTargetRegressor(Ridge(), func=np.log, inverse_func=np.exp))
])

In [201]:
rg_pipe.fit(X_train, y_train)

Pipeline(steps=[('ctx',
                 ColumnTransformer(transformers=[('ss', StandardScaler(),
                                                  ['home_quality', 'home_sqft',
                                                   'garage_cars', 'home_age',
                                                   'yr_remod', 'full_bath',
                                                   'masonry_veneer_area',
                                                   'total_rooms_above_ground']),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['neighborhood',
                                                   'home_type'])])),
                ('rg',
                 TransformedTargetRegressor(func=<ufunc 'log'>,
                                            inverse_func=<ufunc '

In [202]:
print('Final Ridge RMSE (Train):', round(np.sqrt(mean_squared_error(y_train, rg_pipe_poly.predict(X_train))), 2))
print('Final Ridge RMSE (Test):', round(np.sqrt(mean_squared_error(y_val, rg_pipe_poly.predict(X_val))), 2))

Final Ridge RMSE (Train): 20000.34
Final Ridge RMSE (Test): 23380.07


In [203]:
print('Ridge Training Score:', round(cross_val_score(rg_pipe_poly, X_train, y_train).mean(), 3))
print('Ridge Validation Score:', round(cross_val_score(rg_pipe_poly, X_val, y_val).mean(), 3))

Ridge Training Score: 0.894
Ridge Validation Score: 0.847


## Running the Ridge Model on Official Test Data

In [204]:
# Cleaning test data so that the model properly recognizes variables

def clean_df_test(df):
    dict_home_type = { # Creating a dictionary that correlates to the data dictionary and will replace numerics with string values in the 'home_type' / 'MS SubClass' column.
    20 : 'Newer 1 Story', 
    30 : 'Older 1 Story', 
    40 : '1 Story Finished Attic',
    45 : '1.5 Story Unfinished',
    50 : '1.5 Story Finished',
    60 : 'Newer 2 Story',
    70 : 'Older 2 Story',
    75 : '2.5 Story',
    80 : 'Split or Multi-Level',
    85 : 'Split Foyer',
    90 : 'Duplex',
    120 : '1 Story PUD',
    150 : '1.5 Story PUD',
    160 : '2 Story PUD',
    180 : 'PUD Multilevel',
    190 : '2 Family Conversion'
}
    
    df['home_sqft'] = df['Gr Liv Area'] + df['Garage Area'] + df['Total Bsmt SF'] # Creating a new SqFt variable that captures total living area sqft + garage sqft + basement sqft
    df['home_age'] = df['Yr Sold'] - df['Year Built']
    df = df[df['home_age'] >= 0] # removing any rows where home_age might be negative, as that isn't possible and indicates an error in the original data
    df = df[[ # Filtering dataset by most important variables needed
    'Id', # ID of the home. Need this to complete the task.
    'Overall Qual', # Quality of the exterior of the home, rated 1-10.
    'home_sqft', # Sum SqFt of Garage, Living Areas, and Basement
    'Garage Cars', # Garage size by how many cars could fit (e.g. 2-car garage)
    'home_age',
    'Year Remod/Add', # Year remodelled. If not remodelled, same year as Year Built
    'Full Bath', # Number of full baths on the property
    'Mas Vnr Area', # Amount of masonry veneer - exterior decoration (e.g. brick, stone, etc.)
    'TotRms AbvGrd', # Total rooms above the basement.
    'Neighborhood',
    'MS SubClass']]

    df.replace({'MS SubClass' : dict_home_type}, inplace=True) # replacing numeric keys in series with string values in dict_home_type

    df = df.rename(columns={ # renaming columns to snake case
    'Id' : 'id',
    'Overall Qual' : 'home_quality',
    'Garage Cars' : 'garage_cars',
    'Year Remod/Add' : 'yr_remod',
    'Full Bath' : 'full_bath',
    'Mas Vnr Area' : 'masonry_veneer_area',
    'TotRms AbvGrd' : 'total_rooms_above_ground',
    'Neighborhood' : 'neighborhood',
    'MS SubClass' : 'home_type'
    })

    df = df.fillna(0)

    return df


df_test = clean_df_test(df_test)

In [205]:
test_id = df_test['id'] # Storing the id column
df_test = df_test.drop(columns=['id']) # Dropping the id column in df_test

In [206]:
preds = rg_pipe.predict(df_test) # Final prediction
preds = pd.DataFrame(preds, columns = ['SalePrice']) # Create submission dataframe
preds.insert(loc = 0, column = 'Id', value = test_id) # Insert ID column

In [207]:
preds.to_csv('../submissions/ridge_3.csv', index=False) # Final Kaggle RMSE Score: 23638

## Conclusion

The final Ridge model can account for approximately 84% of variance in new, unseen data, and our RMSE on unseen data is 23638. This means that there is an element of risk - the positive and negative range of potential error that the model could predict is up to $23,638 when our model is trying to predict home price. At a higher scale, and especially in today's housing market, this error is minimal, especially beyond the $400,000 range. Also, our model cannot account for 16% of variance in the data. Comparing this to the null RMSE (65897), using our model to predict home price or interpret which variables have a measured effect is going to be much more effective than simply guessing by using the mean home price.